In [79]:
import pandas as pd
import numpy as np
import datetime as dt

In [80]:
base = pd.read_csv('data.csv', encoding="ISO-8859-1")

In [81]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [83]:
base.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [85]:
base = base[base['Quantity'] >= 1]

In [87]:
base.describe()

,Quantity,UnitPrice,CustomerID
count,531285.000000,531285.000000,397924.000000
mean,10.655262,3.857296,15294.315171
std,156.830323,41.810047,1713.169877
min,1.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13969.000000
50%,3.000000,2.080000,15159.000000
75%,10.000000,4.130000,16795.000000
max,80995.000000,13541.330000,18287.000000


In [89]:
base = base[base['UnitPrice'] >= 1]

In [90]:
base.describe()

,Quantity,UnitPrice,CustomerID
count,419211.000000,419211.000000,304911.000000
mean,8.279623,4.776021,15297.045351
std,171.939050,40.342822,1711.528333
min,1.000000,1.000000,12346.000000
25%,1.000000,1.650000,13975.000000
50%,3.000000,2.890000,15153.000000
75%,8.000000,4.950000,16805.000000
max,80995.000000,13541.330000,18287.000000


In [91]:
base['Totalvalue'] = base['Quantity'] * base['UnitPrice']

In [92]:
base = base[['InvoiceDate','InvoiceNo','CustomerID','Totalvalue',]]

In [93]:
base

,InvoiceDate,InvoiceNo,CustomerID,Totalvalue
0,12/1/2010 8:26,536365,17850.0,15.30
1,12/1/2010 8:26,536365,17850.0,20.34
2,12/1/2010 8:26,536365,17850.0,22.00
3,12/1/2010 8:26,536365,17850.0,20.34
4,12/1/2010 8:26,536365,17850.0,20.34
...,...,...,...,...
541903,12/9/2011 12:50,581587,12680.0,16.60
541905,12/9/2011 12:50,581587,12680.0,12.60
541906,12/9/2011 12:50,581587,12680.0,16.60
541907,12/9/2011 12:50,581587,12680.0,16.60


In [94]:
base['InvoiceDate'] = pd.to_datetime(base['InvoiceDate'])

In [95]:
PRESENT = dt.datetime(2011,12,12)

In [96]:
RFM = base.groupby('CustomerID').agg({'InvoiceDate': lambda x: (PRESENT - x.max()).days, 
                                        'InvoiceNo': lambda x: len(x),      
                                        'Totalvalue': lambda x: x.sum(),
                                        }) 

RFM['InvoiceDate'] = RFM['InvoiceDate'].astype(int)
RFM.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'Totalvalue': 'monetary_value'}, inplace=True)

In [97]:
RFM.head()

,recency,frequency,monetary_value
CustomerID,,,
12346.0,327,1,77183.60
12347.0,4,139,3764.48
12348.0,77,11,1114.00
12349.0,20,64,1668.75
12350.0,312,14,295.30


In [98]:
RFM.describe()

,recency,frequency,monetary_value
count,4304.000000,4304.000000,4304.000000
mean,94.262314,70.843634,1827.118146
std,100.313542,176.973765,8358.026915
min,2.000000,1.000000,3.750000
25%,19.000000,13.000000,257.177500
50%,52.000000,32.000000,578.950000
75%,144.250000,78.000000,1431.910000
max,375.000000,5885.000000,257518.900000


In [100]:
quantiles = RFM.quantile(q=[0.2,0.4,0.6,0.8,])

In [101]:
quantiles = quantiles.to_dict()

In [102]:
quantiles

{'recency': {0.2: 14.0, 0.4: 34.0, 0.6: 73.0, 0.8: 182.0},
 'frequency': {0.2: 10.0, 0.4: 23.0, 0.6: 45.0, 0.8: 94.0},
 'monetary_value': {0.2: 210.652,
  0.4: 420.37600000000003,
  0.6: 813.9219999999999,
  0.8: 1782.2240000000002}}

In [103]:
rfmSeg = RFM

In [104]:
def RClass(x,p,d):
    if x <= d[p][0.2]:
        return 5
    elif x <= d[p][0.4]:
        return 4
    elif x <= d[p][0.6]: 
        return 3
    elif x <= d[p][0.8]: 
        return 2
    else:
        return 1
    
def FMClass(x,p,d):
    if x <= d[p][0.2]:
        return 1
    elif x <= d[p][0.4]:
        return 2
    elif x <= d[p][0.6]: 
        return 3
    elif x <= d[p][0.8]: 
        return 4
    else:
        return 5

In [105]:
rfmSeg['R_Quartile'] = rfmSeg['recency'].apply(RClass, args=('recency',quantiles,))
rfmSeg['F_Quartile'] = rfmSeg['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfmSeg['M_Quartile'] = rfmSeg['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [106]:
rfmSeg['RFMClass'] = rfmSeg.R_Quartile.map(str) \
                            + rfmSeg.F_Quartile.map(str) \
                            + rfmSeg.M_Quartile.map(str)

In [107]:
rfmSeg

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CustomerID,,,,,,,
12346.0,327,1,77183.60,1,1,5,115
12347.0,4,139,3764.48,5,5,5,555
12348.0,77,11,1114.00,2,2,4,224
12349.0,20,64,1668.75,4,4,4,444
12350.0,312,14,295.30,1,2,2,122
...,...,...,...,...,...,...,...
18280.0,279,10,180.60,1,1,1,111
18281.0,182,4,65.70,2,1,1,211
18282.0,9,9,152.45,5,1,1,511
